# Lecture 5: Sources of Risk

[intro statement] 

The main topics of discussion are:

 1. Fundamental Risk Factors
 2. Risk Mapping
 



In [1]:
#Importing standard libraries


import numpy as np
from matplotlib import pyplot as plt 
import scipy as sci
import pandas as pd

risks are kinda risky,

thank you.